<a href="https://colab.research.google.com/github/Viksilio/DA_plus/blob/master/SQL_WorkShop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Подключение библиотек

In [ ]:
import pandas as pd
from sqlalchemy import create_engine 

### Конфигурация для подключения к локальной базе данных PostreSQL

In [ ]:
db_config = {
    'user': 'postgres', # имя пользователя
    'pwd': '1202', # пароль
    'host': 'localhost',
    'port': 5432, # порт подключения
    'db': 'WorkShop' # название базы данных
}  

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)

#### Создание подключения

In [ ]:
engine = create_engine(connection_string) 

### Задача 1

В таблице total_sales содержится информация о сделках.  
Требуется написать SQL запрос, который к каждой транзакции добавит:
* Дату первой сделки покупателя.
* Дату предыдущей сделки покупателя.
* Покажет общее количество товаров, купленных покупателем.
* Присвоить всем сделкам покупателя / продавца порядковый номер.

In [ ]:
## Решение
query='''
select *,
first_value(date) over (partition by buyer_id order by date),
LAG(date) over (partition by buyer_id order by date),
SUM(qty) over (partition by buyer_id),
DENSE_RANK() over (order by buyer_id, sales_id)
from total_sales
order by sales_id
'''

# Проверка данных
first = pd.read_sql_query(query, con=engine) 
first

,sales_id,date,buyer_id,seller_id,qty,qty_shipped,first_value,lag,sum,dense_rank
0,25651,2022-06-11,4,f,40,NaN,2022-06-11,None,110,5
1,34489,2022-06-18,1,f,30,NaN,2022-04-12,2022-04-12,40,1
2,36044,2022-07-07,2,g,10,10.0,2022-07-06,2022-07-06,80,3
3,40494,2022-04-12,1,m,10,10.0,2022-04-12,None,40,2
4,83056,2022-07-06,2,m,70,NaN,2022-07-06,None,80,4
5,96498,2022-07-19,4,e,70,70.0,2022-06-11,2022-06-11,110,6


### Задача 2

#### Запрос 1


Схема базы данных  
![image.png](https://lh3.googleusercontent.com/3xRB7AUIuQqHDMkKsZHUPZvWfQbeHe3a1f6EeOw4P97jDsok5koPEV1esYd83vnw8TODB6BmtWIhKqCNMbm7fXOSF_e4qJxlk9PMtobwz-METle6xyO4e2cnehebRAlWuuzCzAC3MgQggjylFrwI2a9HJ4bZqTJ5Dqss-YQAs2EcIBg99YP2m31VP8vGNlGSyMMz0NlvQ5p0K0I4IXLlKdo_YWLK5l8Trsx6K7KjR9xumje_zzUB3XjVcdj78hwInziuGr9T1OJsq-Ix10QygkAZrj5i_bmaBN73mruvRjBpbxpncQtNhTLzOlEnWMxOH8gY2OOWeZT9G18GISPv_tt14n29H8J77S5eL-5T-PK2a0bElH0ptj0ygxbiYUSvrF3xaCcfVIMp2iomfHlQ0YlePPDSaRYrZet8IQ32J7ZiWjl5ELxCVTWYxi6BPA7ek_NcIX6HOhEFyaagPwfceYDdI7DWZEBhOH7LofGFcCpM4FpYyCw-Mbe34DfYgkL63Mhfspo7z2iQnFnHlOpA_cKcqtLlSqasWi6rMCQBg1IC5wapT7phwHDAec3q5IJCs_r4h4L_a9OrixxYkJqt94dJVvTTL9gBWD5X83iwIv-H4cLB52fk7fRG1F4RbPeUIIOqMDaNPLzSh3TezADzs74cr3uQPScS494Wlfxg287uLn_4jIgys7iF0QHxKY7gBCVV5iiXsbpTunYnM-61gItD7kT0eUXo0m1S302JeQOs18ahK7vvLEhNZICJjthuraf3LTFu2kfNPj7vGXDymYawGRr_M2uMUjbhmnPUS-eQLuAiapPPA3j8odl4kBZgNcx31ynX_gor4y2zC4ihcFeubZhsM-msxP6LcpgtQqmr_3gaYL7z8LIKCU1iZ0b3DO91KW7rMWB0CUsMjliVbdPA6nWKGcG2ChwP0E9pjGuW=w460-h223-s-no?authuser=0)

Составьте запросы к базе, отвечающие на вопросы:  
* Для каждого покупателя найдите магазин, в котором он совершил
наибольшее количество покупок.  
* Найдите всех покупателей, которые в каждом полном месяце этого года
совершали не менее 3 покупок.  

In [ ]:
## Решение
query='''
with count_sales as (
select count(s.sales_id) over(partition by customer_id, s.store_id) as cnt, s.store_id, s.customer_id
from sales s
order by cnt desc, customer_id desc)
select distinct store_nm, customer_id
from (select first_value(store_id) over (partition by customer_id order by cnt desc) as store_id, customer_id 
     from count_sales) ss
left join stores st on st.store_id = ss.store_id 
order by customer_id
'''

# Проверка данных
second_1 = pd.read_sql_query(query, con=engine) 
second_1

,store_nm,customer_id
0,store 1,1
1,store 1,2
2,store 2,3
3,store 3,4
4,store 1,5
5,store 1,6
6,store 1,7


In [ ]:
## Решение
query='''
select distinct customer_id from (
select distinct customer_id, date_trunc('month', dt), count(sales_id)
from sales s where dt between '01.01.2023' and '28.02.2023'
group by date_trunc('month', dt), customer_id
having COUNT (sales_id) >= 3) aa
order by customer_id
'''

# Проверка данных
second_2 = pd.read_sql_query(query, con=engine) 
second_2

,customer_id
0,1
1,2
2,3


#### Запрос 2

Напишите запрос, формирующий ниже представленный отчет

![image.png](https://lh3.googleusercontent.com/-ust2NrpSsTrtjt2RijwfaKL8sIUHTcSmhPwPDNPLIMnjoh-j0bdT-ud3e6U-FqqUTTkl2Vq0AK-UYjHtsW9RyKtOxV00awGBgMR-BSUHkhZW4HCc8zEKuDrh8JUBJ8g9g-wrT2TPzQCROa-jYJc6NmYD1KgXBvJhK31v7T7AdZyIOtLtyutjNCAVUsKAXyjpyCK8yvPXwHPNuIoV8Ih_QHRIcTmvsNMJ357h5WOy2pTXJ1UFdoKEG83o5VGYGIaOKgril0NbG5Ks1PIK_hni_qk7VBf32DXk4PvNpyavEULwXroNrN_0Xl59-zbgxVGzib-_QABR0yHC_0lvwaTWD4lqlVxJJ6s5fdqWdC41vts0bnf9GgaXbWstmhZoNWn352KVxz1Dt-94zV4BZYMmA7yygWcmVii_QXZkeonf68sg0nmfLTso3A8WyoNn8irYSToAb348O_WuJvr-2IS0kE0fxBfsMOgQpka0ss9IIkUHM-IN3WiiouH3sbPqyuG7mMgh0LW-ZOxjSm07k6TmdseLsT8EzDHe7FPxqtUUxo2AglF2-Jot42dHqes9w-hw3Qz3D-yOxB_Po4Nv_uxwsGBCUkfVhAtsSZqgDCigdNFaQ_6XHSzJvENrawvQufv2TBR3-ZjL9aw_V46uD8oJ5_gJKjqG231aGfr7a1ONN5bXaQlbeuv8KtmkOmRrbzogXjfstfRGwEK5n-GbFmgC9NJROZraud7etFPoJTzta7q4P2JE7gmFGG0LUNCfuOp9R_xIcooTrzLEzq5_uYZPbeRzmk8lkev0yruBnLEiZTJ1x395R-PH3WpJw-exdiL9PA11KNzygXmLud0k6c2qMoN9XS46xohT-pt8qo4bmwSIeOZJw0BKd1Sn82pv8zcjMFrqrIr_FATGymVf1xzK0LdkrM-LUvOAnbKpnHjjd2x=w464-h104-s-no?authuser=0)

In [ ]:
## Решение
query='''
SELECT (date_trunc('MONTH', s.dt) + INTERVAL '1 MONTH - 1 day')::date as month_end_dt,
SUM(s.amt) over (partition by (date_trunc('MONTH', dt) + INTERVAL '1 MONTH - 1 day')::date) as company_sales_amt,
st.store_nm,
SUM(s.amt) over (partition by (date_trunc('MONTH', dt) + INTERVAL '1 MONTH - 1 day')::date, st.store_nm) as store_sales_amt,
round(SUM(s.amt) over (partition by (date_trunc('MONTH', dt) + INTERVAL '1 MONTH - 1 day')::date, st.store_nm)/SUM(s.amt) over (partition by (date_trunc('MONTH', dt) + INTERVAL '1 MONTH - 1 day')::date), 2) as store_sales_share
from sales s
left join stores st on s.store_id = st.store_id 
'''

# Проверка данных
second_3 = pd.read_sql_query(query, con=engine) 
second_3.head()

,month_end_dt,company_sales_amt,store_nm,store_sales_amt,store_sales_share
0,2023-01-31,1400.0,store 1,660.0,0.47
1,2023-01-31,1400.0,store 1,660.0,0.47
2,2023-01-31,1400.0,store 1,660.0,0.47
3,2023-01-31,1400.0,store 1,660.0,0.47
4,2023-01-31,1400.0,store 1,660.0,0.47


### Задача 3

#### Запрос 1

В таблице calls хранятся данные всех звонков абонентов за отчетный месяц.  
В поле roaming указан факт нахождения абонента в роуминге во время звонка:
* 1 - абонент был в роуминге;
* 0 - абонент не был в роуминге.

НапишитеSQL запрос, определяющий уникальные id абонентов, которые хоть раз
звонили в роуминге за отчетный месяц.

In [ ]:
## Решение
query='''
select distinct id_abon
from calls c 
where roaming = 1
order by id_abon 
'''

# Проверка данных
third_1 = pd.read_sql_query(query, con=engine) 
third_1.head()

,id_abon
0,1
1,4
2,6
3,7
4,8


#### Запрос 2

В таблице abnt хранятся уникальные id всех абонентов, совершавших звонки за
отчетный месяц.  
В таблице roam хранятся уникальные id абонентов, которые
были в роуминге за отчетный месяц.  
Напишите SQL запрос, результатом которого будут все id абонентов, которые не
были в роуминге в текущем месяце.

In [ ]:
## Решение
query='''
select distinct id_abon
from abnt a 
where id_abon not in(
select id_abon
from roam r)
'''

# Проверка данных
third_2 = pd.read_sql_query(query, con=engine) 
third_2.head()

,id_abon
0,6
1,1


#### Запрос 3

В таблицах abnt и roam также хранятся данные о тратах абонентов. В таблице
abnt есть все начисления абонентов, в таблице roam начисления в роуминге.  
Напишите SQL запрос, который выведет всех абонентов с начислениями в
домашней сети больше 0.  
Начисления в домашней сети= все начисления (abnt.clc)  - начисления в роуминге
(roam.clc).

In [ ]:
## Решение
query='''
select a.id_abon, a.clc - r.clc as clc
from abnt a 
join roam r on r.id_abon = a.id_abon
order by clc desc
'''

# Проверка данных
third_3 = pd.read_sql_query(query, con=engine) 
third_3.head()

,id_abon,clc
0,4,300
1,2,0
2,3,0
3,5,0
4,7,0


#### Запрос 4

Используя данные таблицы calls, выведите id всех абонентов, у которых было
больше 5 звонков в роуминге.

In [ ]:
## Решение
query='''
select distinct id_abon
from calls c 
where roaming = 1
group by id_abon 
having count(roaming) > 5
'''

# Проверка данных
third_4 = pd.read_sql_query(query, con=engine) 
third_4.head()

,id_abon
0,1


#### Запрос 5

Используя данные таблицы calls, выведите периоды нахождения абонентов в
роуминге. Время пребывания в роуминге определяется непрерывной последовательностью звонков в роуминге.

In [ ]:
## Решение
query='''
with ol as (select id_abon,
date as day,
lag(date) over (partition by id_abon order by date_trunc('day',date)) as prev_day,
lead(date) over (partition by id_abon order by date_trunc('day',date)) as next_day
from calls c
where roaming = 1
)
select  distinct id_abon, case
when prev_day+1 = day then concat(to_char(prev_day, 'dd.MM'),'-',to_char(day, 'dd.MM'))
when next_day-1 = day then concat(to_char(day, 'dd.MM'),'-',to_char(next_day, 'dd.MM'))
else NULL
end as period_on_roam
from ol
'''

# Проверка данных
third_5 = pd.read_sql_query(query, con=engine) 
third_5.head()

,id_abon,period_on_roam
0,1,03.01-04.01
1,1,04.01-05.01
2,1,05.01-06.01
3,1,None
4,4,None


### Задача 4

У вас есть две таблицы:
* t1 - данные по заявкам на Подбор кредита с сайта Сравни.ру;
* t2 - выгрузка банка о решениях по выдаче кредита.
Напишите SQL-запрос, содержащий всех пользователей, получивших кредит.

![image.png](https://lh3.googleusercontent.com/4bb6r_ryA-3WctBwsiAWyd-hY-MST21SRbidHgEGVh40FxujJzVqySQ_ZdK-bT9NAyYW2Sqf3RYxIfe12zMgDgA_VAMPmt5xqqiiOeVuoxr4fzlJBkYTQRvSZ9vJKf1WOWircFENlMG2Eyxlp0bydQhpORmLBUJ2Mg_0j8LNylJ9TO5lFYpmUW6gSX6c6x4h-FW-KBvSttPWUDAHxFtNTMNOGlX_kJucMX9cvbCaz-Md_C4KLPAV0K3ps9RSiCDbJnMVKkglKoqXwqbJo9TXS6j0KkLSCCaU7CTqgPTJ57UIswNR2b3M2O1vkjR9QRlsfU4O3r7OS_9Kjq03wrc_LfsMU__znQ0vb9mdLwxtNOJUJrsWMvICbnZNzEy2zBdyxjvTMsl3dhkFWZe8HrT4tBBsiIo82q6E_UPbFpcjj7aM3RAzOArGlqYv18xmvlv-TehoU9m5tF8M5SqCgs_39Qx1A13rm_H09UWjCU8hokDbMK9wk_VWOs63Ap5HQPrsU0nGUbMQb2OS_6A2nzykcGtvX8JvGBNiUrGENnzWeLWc2YRy4CVnrESldJSQFRzFkprWx22106OX2gQN2OKsoRWWE85aNpovMmcYAdcq46l-P_mVK1foFLV7D5udRr2uf-ia2Wb08aJPZgMRCuwqR8KzVcPNWqnBGtrVFCAoe3pR2a2DUI4N5-3v9PPskJhlDCs32I9NiqglYNVQzEz5CgZ0V0viJxAyVLQqoaYnawn5O4WXO_TJgzcUbC-5AxQOWul9Yzr18MEZbTW-2rTJn4LonwxslqhQ0NZsiwsd8Y-0_hmSVR00Ps2meXRyCCMYBdwFCJLuLfc5sJTRbRX1eII4iCExPLpzgfs7FXtCzSscZ6YFnxl6YCXarSSlmMStjuPvqyKsx3Ud_wavdc0LrKTiKNITct_kcm-Z-DpHle7u=w469-h290-s-no?authuser=0))

In [ ]:
## Решение
query='''
select t1.*, t2.status 
from t1
left join t2 on t2.id = t1.id
where t2.status = 'выдан'
'''
## лучше вариант с "where t2.status like '%выд%'"

# Проверка данных
fourh = pd.read_sql_query(query, con=engine) 
fourh.head()

,id,surname,birthday,amount,salary,status
0,77664,Мамин-Сибиряк,2003-01-20,6000000,58000,выдан
1,52183,Васильев,1956-07-05,750000,98000,выдан
